In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# -*- coding: utf-8 -*-
"""alexnet_005_mixup
Back to Basics + Mixup Augmentation
"""

import kagglehub
import os

# ============================================
# 0. 데이터셋 및 설정
# ============================================


import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import GroupKFold
from PIL import Image
from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

# Seed 고정
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 경로 설정
train_dir = "/kaggle/input/state-farm-distracted-driver-detection/imgs/train"
test_dir  = "/kaggle/input/state-farm-distracted-driver-detection/imgs/test"
driver_csv_path = "/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv"

# 하이퍼파라미터
img_size = 224
batch_size = 32  # Mixup은 배치가 클수록 좋지만 GPU 한계 고려
num_classes = 10
num_epochs = 40
learning_rate = 1e-4 # Mixup 사용 시 학습이 조금 더디므로 LR 조절
num_workers = 2
patience = 8 # Mixup은 Loss가 진동하므로 patience를 넉넉히 줌

# ============================================
# 1. Mixup 함수 정의 (핵심)
# ============================================
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size(0)
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# ============================================
# 2. Dataset & Transform
# ============================================
class DriverDataset(Dataset):
    def __init__(self, img_paths, labels=None, transform=None, is_test=False):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        path = self.img_paths[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        if self.is_test:
            return img, os.path.basename(path)
        return img, self.labels[idx]

def get_data_lists(df, img_dir):
    img_paths = []
    labels = []
    for _, row in df.iterrows():
        c_name = row["classname"]
        path = os.path.join(img_dir, c_name, row["img"])
        img_paths.append(path)
        labels.append(int(c_name[1:]))
    return img_paths, labels

# Augmentation 강화
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)), # 스케일 조정
    transforms.RandomHorizontalFlip(p=0.0), # 운전석 위치 바뀌면 안됨! 절대 금지
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ============================================
# 3. 데이터 로드 (Fold 0)
# ============================================
df = pd.read_csv(driver_csv_path)
# [수정 코드] Fold 번호를 바꿔가며 실행 가능
TARGET_FOLD = 1 # <--- 실행할 때마다 0, 1, 2, 3, 4로 변경!

gkf = GroupKFold(n_splits=5)
folds = list(gkf.split(df, df["classname"], df["subject"]))
train_idx, val_idx = folds[TARGET_FOLD] # next() 대신 인덱싱 사용

train_df = df.iloc[train_idx]
val_df = df.iloc[val_idx]

train_paths, train_labels = get_data_lists(train_df, train_dir)
val_paths, val_labels = get_data_lists(val_df, train_dir)

train_ds = DriverDataset(train_paths, train_labels, transform=train_transform)
val_ds = DriverDataset(val_paths, val_labels, transform=val_transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# ============================================
# 4. 모델 및 학습 (Mixup 적용)
# ============================================
# AlexNet Full Unfreeze (003 방식 복귀)
model = models.alexnet(weights="IMAGENET1K_V1")
model.classifier[6] = nn.Linear(4096, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

best_loss = 999
best_model_path = "best_alexnet_mixup.pth"
es_counter = 0

print("🚀 Training Start with Mixup...")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    # Mixup Training Loop
    for x, y in tqdm(train_loader, desc=f"Ep {epoch+1} Train", leave=False):
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        
        # 🔥 Mixup 적용
        inputs, targets_a, targets_b, lam = mixup_data(x, y, alpha=0.4, use_cuda=True)
        outputs = model(inputs)
        loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x.size(0)
        
    avg_train_loss = total_loss / len(train_ds)
    
    # Validation (Mixup 없이 정상 평가)
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="Val", leave=False):
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item() * x.size(0)
            correct += (out.argmax(1) == y).sum().item()
            
    avg_val_loss = val_loss / len(val_ds)
    val_acc = 100 * correct / len(val_ds)
    
    scheduler.step(avg_val_loss)
    
    print(f"Ep {epoch+1} | Train Loss {avg_train_loss:.4f} (Mixup) | Val Loss {avg_val_loss:.4f} | Acc {val_acc:.2f}%")
    
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        es_counter = 0
        print(f"🔥 Best Model Saved: {best_loss:.4f}")
    else:
        es_counter += 1
        if es_counter >= patience:
            print("⛔ Early Stopping")
            break

# ============================================
# 5. 예측 (FiveCrop TTA 유지)
# ============================================
print("\nPredicting with TTA...")
model.load_state_dict(torch.load(best_model_path))
model.eval()

tta_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.FiveCrop(224),
    transforms.Lambda(lambda crops: torch.stack([
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(transforms.ToTensor()(crop)) 
        for crop in crops
    ]))
])

test_files = sorted(os.listdir(test_dir))
test_paths = [os.path.join(test_dir, f) for f in test_files]
test_ds = DriverDataset(test_paths, transform=tta_transform, is_test=True)
test_loader = DataLoader(test_ds, batch_size=batch_size//2, shuffle=False, num_workers=num_workers)

preds = []
names = []

with torch.no_grad():
    for x, fnames in tqdm(test_loader):
        bs, ncrops, c, h, w = x.size()
        x = x.view(-1, c, h, w).to(device)
        out = model(x)
        out = out.view(bs, ncrops, -1).mean(1)
        prob = torch.softmax(out, dim=1).cpu().numpy()
        preds.append(prob)
        names.extend(fnames)

preds = np.vstack(preds)
submission = pd.DataFrame({
    "img": names,
    **{f"c{i}": preds[:, i] for i in range(10)}
})
submission.to_csv("submission_alexnet_mixup_fold1.csv", index=False)

from IPython.display import FileLink
display(FileLink('submission_alexnet_mixup_fold1.csv'))